<h1>Sentiment analysis</h1>
<ol>
<h3><li>Définition du Sujet et Objectif</h3></li>
<b>Sentiment analysis ?</b><br>
<p>Aussi connue sous le nom de <strong><i>« Opinion Mining »</i></strong>, <strong>l’analyse des sentiments</strong> consiste à identifier les informations subjectives d’un texte pour <strong>extraire l’opinion de l’auteur</strong>. <br>
De manière générale, <strong>l’analyse des sentiments</strong> permet de mesurer le niveau de satisfaction des clients vis-à-vis des produits ou services fournis par une entreprise ou un organisme. Elle peut même s’avérer <strong>bien plus efficace que des méthodes classiques</strong> comme les sondages puisque <strong>de nos jours, une partie croissante des consommateurs partage fréquemment leurs opinions sur les réseaux sociaux</strong>. </p>

<p><i><strong>Objectif du Projet:</strong></i> Notre but est donc de mettre en place un modèle permettant de classifier les différents avis des clients d’un hôtel en deux classes qui sont : Avis positifs et avis négatifs.<br>
Cette classification permettra au personnel de l’hôtel de pouvoir identifier les principales plaintes (Texte négatifs) afin d’améliorer leurs services et réduire le niveau d’insatisfaction des clients.<br></p>

<h3><li>Constitution de la dataset</h3></li>
<p>Avant d’entamer le travail proprement dit, nous allons d’abord décrire brièvement notre dataset.<br>
Nous avons utilisé une dataset téléchargeable sur Kaggle via le lien suivant : 
<a href="https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews">https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews</a><br></p>


<p>Aperçu de la dataset</p>

In [2]:
import pandas as pd
df = pd.read_csv('data.csv')

df.to_csv('data.csv', index=False)  

df.head()


,Review,Rating
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
2,nice rooms not 4* experience hotel monaco seat...,1
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1


En affichant le résumé de la dataset, on voit qu’elle contient 15 000 lignes et deux colonnes : <br>
<ul>
    <li>Une colonne Review : qui n’est autre qu’un paragraphe contenant le commentaire de l’utilisateur</li>
    <li>Une colonne Rating : qui précise s’il s’agit d’un avis positif ou pas. Notre objectif principal est de détecter les avis négatifs donc la classe positive correspond aux avis négatifs qui et la classe négative correspond aux avis positifs.</li>
</ul>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


<p>En affichant les quantités de données pour chaque catégorie, on s’aperçoit qu’elle est disproportionnée. Elle contient plus d’avis positifs (environ 17000) que d’avis négatifs (environ 3000). Cette différence pourra sans doute influencer le modèle lors de l’entrainement. Ce dernier risque donc d’overfitter sur les avis positifs.</p>

In [4]:
df.Rating.value_counts()

1    17277
0     3214
Name: Rating, dtype: int64

Pour remédier à cela, il existe plusieurs possibilités :
<ul>
    <li><b>Sous-échantillonnage de la classe majoritaire</b> qui consiste à réduire le nombre d'échantillons de la classe majoritaire en sélectionnant au hasard un sous-ensemble de points de données de cette classe à utiliser pour la formation.</li>
    <br>
    <li><b>Suréchantillonnage de la classe minoritaire </b>qui consiste à augmenter le nombre d'échantillons de la classe minoritaire dans l'ensemble de données d'apprentissage. La méthode courante consiste à ajouter des copies de points de données de la classe minoritaire, ce qui amplifie la région de décision, ce qui améliore les métriques d'évaluation.</li>
     <br>
    <li>Faire du <b>webscraping</b> afin d’augmenter le nombre d’avis négatifs.</li>

</ul>


L'un des principaux inconvénients du sous-échantillonnage est que des données ou des informations utiles peuvent être perdues. De plus dans notre cas, on obtiendra en tout environ 6000 lignes de données, ce qui n’est pas vraiment approprié pour un problème de NLP.<br><br>
Le principal inconvénient du suréchantillonnage est qu'elle peut entraîner un overfitting. <br><br>
Nous allons donc opter pour la troisième solution qui est celle du webscraping.<br><br>
Nous avons scrapper plusieurs sites différents. Les détails du webscraping se trouve dans le notebook <a href="scraping_data.py"> webscraping.ipynb</a>. <br><br>
Grâce au webscraping on a pu obtenir environ 5500 lignes avis négatifs. La dataset contient désormais 25000 données. Cette fois ci, en faisant un sous-échantillonnage, on obtient 17000 données en tout (8500 pour chaque catégorie)<br>


Une fois que cela est fait, Notre dataset est constitué et on peut enchainer avec les étapes suivantes.


<ol start="3">
<h3><li>Plan de travail</h3></li>
Comme on peut s’en douter, Il s’agit ici d’un problème de Natural Language Processing (NLP) ou Traitement automatique du langage naturel.<br>
Et comme pour tout problème de NLP, Notre travail se compose principalement de deux parties :

<ul>
    <li>La partie <b>« linguistique »</b>, qui consiste à prétraiter et transformer les informations en entrée en un jeu de données exploitable.</li>
    <li>La partie <b>« apprentissage automatique »</b>, qui porte sur l’application de modèles <b><i>de Machine Learning</i></b> à ce jeu de données.</li><br>
</ul>
Dans la suite du rapport, Nous allons aborder ces deux aspects, en décrivant brièvement les <b>principales méthodes utilisées </b>et en précisant les principaux défis auxquels nous avons fait face.
</ol>
<ol type="A">
<h3><li>Partie linguistique : Du texte à la donnée</li></h3>
<p>Parmi les principales étapes de cette partie, on retrouve :</p>
<ol type="a">
<li>Nettoyage : cette phase consiste à réaliser des tâches telles que la <b><i>suppression d’urls, d’emoji, suppression des chiffres, ponctuation, symboles et stopwords, passage en minuscule</i><b>.</li>
</ol>
</ol>


Ci-dessous les fonctions permettant de faire le nettoyage.

In [8]:
import nltk
import re
import string
from nltk.corpus import stopwords

def remove_URL (text):
    url = re.compile(r"https?://\s+ www\.\s+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
         

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (i0s)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)


def remove_punct (text):
    table = str.maketrans ("","",string.punctuation)
    return text.translate(table)

stop = set (stopwords.words ( "english"))
def remove_stopwords (text):
    text = [word.lower () for word in text.split() if word.lower() not in stop]
    return " ".join (text)

ModuleNotFoundError: No module named 'nltk'

<p>On regroupe toutes ses fonctions en une seule appelée <i>nettoyerDataframe</i></p>

In [ ]:
def nettoyerDataframe(df):
    Review_cleaned = df.Review.map(lambda x: remove_URL(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_html(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_emoji(x))
    Review_cleaned = Review_cleaned.map(lambda x: remove_punct(x)) 
    Review_cleaned = Review_cleaned.map(remove_stopwords)   
    df.insert(1, "Review_cleaned", Review_cleaned)
    return df

Ensuite on applique cette fonction pour nettoyer la colonne « Review » et on stocke le résultat dans la colonne <i>« Review_cleaned »</i>


In [ ]:
df = nettoyerDataframe(df)

df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))

X = tfidf_vectorizer.fit_transform(df.Review_cleaned)

[ x for x in X.todense()[0][0:].tolist()[0] if x != 0] 

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score

def afficherMetriques(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    print(f"Accuracy : {accuracy * 100} %")
    print (f"Précision {precision * 100} %") 
    print (f"Recall {recall * 100} %") 

In [ ]:
from sklearn.model_selection import train_test_split

y = df["Rating"].values

X_train , X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [ ]:
#Regression logistique
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(multi_class='multinomial')
logisticRegression.fit(X_train, y_train)

y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Regression logistique ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#Naives Bayes
from sklearn.naive_bayes import GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X_train.toarray(),y_train)

y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Naives Bayes ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#SVM
from sklearn import svm

svmModel = svm.SVC(kernel='linear') # Linear Kernel
svmModel.fit(X_train, y_train)

y_pred = svmModel.predict(X_test)

print("\n***** Métriques d'évaluation : SVM ******")
afficherMetriques(y_test,y_pred)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier()
pac.fit(X_train, y_train)

ypred = pac.predict(X_test)

print("\n***** Métriques d'évaluation : PAC ******")
afficherMetriques(y_test,y_pred)

In [ ]:
#Validation croisée sur le modèle de regression logfistique
from sklearn.model_selection import cross_val_score

rappels = cross_val_score(logisticRegression, X,y, cv=10,scoring='recall')
rappels = pd.Series(rappels)
print("Rappels : min=",rappels.min(),"  max=",rappels.max(),"  moyenne=",rappels.mean())

precisions = cross_val_score(logisticRegression, X,y, cv=10,scoring='precision')
precisions = pd.Series(precisions)
print("Precisions : min=",precisions.min(),"  max=",precisions.max(),"  moyenne=",precisions.mean())

accuracys = cross_val_score(logisticRegression, X,y, cv=10,scoring='accuracy')
accuracys = pd.Series(accuracys)
print("Accuracy : min=",accuracys.min(),"  max=",accuracys.max(),"  moyenne=",accuracys.mean())